In [1]:
import pandas as pd
import numpy as np
import os
import re
import random
import pandas as pd

In [8]:
import pandas as pd

chunk_size = 10000
chunks = pd.read_csv("C:/Users/nr802/Downloads/GenAI Projects/aws_review_sofware_dataset.csv", sep=',', chunksize=chunk_size)

# Get the first chunk and access its columns
df = next(chunks)
print(df.columns)


Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')


In [9]:
df.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

In [10]:
df["words"]="default value"
df["sentences"]="default value"


for i in range(df.shape[0]):
    df.at[i,"words"]= list("")
    df.at[i,"sentences"] = list("")


In [11]:
from nltk.tokenize import sent_tokenize

In [12]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\nr802/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
for i in range(499):
    l1= sent_tokenize(df.loc[i,"reviewText"])
    df.at[i,"sentences"]=l1

In [8]:
!pip install pywsd

In [14]:

import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nr802/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [15]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\nr802/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [17]:
from pywsd.utils import lemmatize_sentence

In [18]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\nr802/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
for k in range(df.shape[0]):
    df.at[k,"words"]=list("")
    for j in range(len(df.loc[k,"sentences"])):
        df.at[k,"words"].extend(lemmatize_sentence(df.loc[k,"sentences"][j]))

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger_eng[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger_eng/[0m

  Searched in:
    - 'C:\\Users\\nr802/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\nr802\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
******************************************

In [20]:
df["words_sentences"] = "default"

In [21]:
import functools

# Iterate through the DataFrame
for k in range(df.shape[0]):
    words = df.loc[k, "words"]
    # Check if words is empty and handle accordingly
    if words:
        # Join the words into a single string
        df.loc[k, "words_sentences"] = functools.reduce(lambda a, b: str(a) + " " + str(b), words)
    else:
        # If the list is empty, set the column to an empty string or some default value
        df.loc[k, "words_sentences"] = ""


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df.words_sentences)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [18]:
dense = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf_vectorizer.get_feature_names_out())

In [19]:
df_y=df["verified"]

In [20]:
from sklearn.preprocessing import LabelEncoder

In [21]:
le=LabelEncoder()

In [22]:
df_y_1=pd.DataFrame(df_y)

In [23]:
df_y_enc=df_y_1.apply(le.fit_transform)

In [24]:
df_y_enc.head(5)

,verified
0,1
1,1
2,1
3,1
4,0


In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Random Forest Classifier
rf = RandomForestClassifier(n_estimators=500, random_state=42)
rf.fit(dense, df_y_enc)

# Accuracy
accuracy_rf = rf.score(dense, df_y_enc)
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Accuracy: 64.15%


In [27]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(dense, df_y_enc)

# Compute accuracy
accuracy_nb = nb.score(dense, df_y_enc)
print(f"Naive Bayes Accuracy: {accuracy_nb * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Naive Bayes Accuracy: 63.64%


In [28]:
from sklearn.ensemble import GradientBoostingClassifier

In [29]:
GBC=GradientBoostingClassifier(n_estimators=1000)

In [30]:
gb_c = GBC.fit(dense, df_y_enc)




/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [32]:
gbc_score=GBC.score(dense, df_y_enc)
print(f"gbc_score: {gbc_score* 100:.2f}%")

gbc_score: 64.15%
